In [ ]:
### my helper functions

import sqlite3

def db_query(db_cursor, db_query_as_text):
    db_cursor.execute(db_query_as_text)
    return db_cursor.fetchall()

def display_schema(db_cursor, table_name=None):
    if table_name:
        QUERY = "select sql from sqlite_master where type = 'table' and name = '%s';" % table_name
    else:
        QUERY = "select sql from sqlite_master where type = 'table';"
    return db_query(db_cursor, QUERY)

def list_tables(db_cursor):
    db_cursor.execute('select name from sqlite_master')
    return [r[0] for r in db_cursor
            if not r[0].startswith('sqlite')
            and not r[0].endswith('idx')]

def empty_db(db_input):
    ''' dropping all existing tables and freeing disk memory'''
    if isinstance(db_input, sqlite3.Connection):
        c = db_input.cursor()
    elif isinstance(db_input, sqlite3.Cursor):
        c = db_input
    for table_name in list_tables(c):
        print 'dropping table %s' % table_name
        db_query(c, 'DROP TABLE %s' % table_name)
    print 'Freeing memory on disk'
    db_query(c, 'VACUUM')
    return None


def create_tables(c):
    QUERY = '''
    CREATE TABLE IF NOT EXISTS nodes
    (
        id INTEGER PRIMARY KEY NOT NULL,
        lat REAL,
        lon REAL,
        user TEXT,
        uid INTEGER NOT NULL,
        version TEXT,
        changeset INTEGER,
        timestamp TEXT,
        FOREIGN KEY (uid)  REFERENCES ways (uid),
        FOREIGN KEY (user) REFERENCES ways (user)
    );
    '''
    db_query(c, QUERY)

    QUERY = '''
    CREATE TABLE IF NOT EXISTS node_tags
    (
        id INTEGER NOT NULL,
        key TEXT,
        value TEXT,
        type TEXT,
        valid TEXT,
        FOREIGN KEY (id) REFERENCES nodes (id)
    );
    '''
    db_query(c, QUERY)

    #['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
    QUERY = '''
    CREATE TABLE IF NOT EXISTS ways
    (
        id INTEGER PRIMARY KEY NOT NULL,
        user TEXT,
        uid INTEGER NOT NULL,
        version TEXT,
        changeset INTEGER,
        timestamp TEXT,
        FOREIGN KEY (uid)  REFERENCES nodes (uid),
        FOREIGN KEY (user) REFERENCES nodes (user)
    );
    '''
    db_query(c, QUERY) 

    QUERY = '''
    CREATE TABLE IF NOT EXISTS way_tags
    (
        id INTEGER,
        key TEXT,
        value TEXT,
        type TEXT,
        valid TEXT,
        FOREIGN KEY (id) REFERENCES way (id)
    );
    '''
    db_query(c, QUERY)

    QUERY = '''
    CREATE TABLE IF NOT EXISTS way_nodes
    (
        id INTEGER NOT NULL,
        node_id INTEGER,
        position INTEGER,
        FOREIGN KEY (id)  REFERENCES ways (id),
        FOREIGN KEY (node_id) REFERENCES nodes (id)
    );
    '''
    db_query(c, QUERY)